In [1]:
import pandas as pd
import requests

In [2]:
#install for sentiment analysis
!pip install textblob

In [3]:
#install corpora
!python -m textblob.download_corpora

Finished.


[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\12039\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\12039\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\12039\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\12039\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to
[nltk_data]     C:\Users\12039\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\12039\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!

In [4]:
# Read location info and listings CSV files into pandas DataFrames
location_costs = pd.read_csv('https://raw.githubusercontent.com/kflemming30/6505-Project/main/cost_of_living_us.csv')

airbnb_property_reviews = pd.read_csv('https://raw.githubusercontent.com/kflemming30/6505-Project/main/Airbnb_Open_Data.csv',
    dtype=str) 
seattle_listings = pd.read_csv('http://data.insideairbnb.com/united-states/wa/seattle/2023-09-18/visualisations/listings.csv')

In [5]:
location_costs.head()

,case_id,state,isMetro,areaname,county,family_member_count,housing_cost,food_cost,transportation_cost,healthcare_cost,other_necessities_cost,childcare_cost,taxes,total_cost,median_family_income
0,1,AL,True,"Montgomery, AL MSA",Autauga County,1p0c,8505.72876,3454.91712,10829.16876,5737.47984,4333.81344,0.0000,6392.94504,39254.0532,73010.414062
1,1,AL,True,"Montgomery, AL MSA",Autauga County,1p1c,12067.50240,5091.70788,11588.19288,8659.55640,6217.45896,6147.8298,7422.07836,57194.3256,73010.414062
2,1,AL,True,"Montgomery, AL MSA",Autauga County,1p2c,12067.50240,7460.20308,12361.77720,11581.63260,7075.65816,15824.6940,9769.56228,76141.0308,73010.414062
3,1,AL,True,"Montgomery, AL MSA",Autauga County,1p3c,15257.15040,9952.23924,13452.18600,14503.70760,9134.35620,18802.1892,13101.70320,94203.5328,73010.414062
4,1,AL,True,"Montgomery, AL MSA",Autauga County,1p4c,15257.15040,12182.21400,13744.59840,17425.78560,9942.36396,18802.1892,13469.21880,100823.5200,73010.414062


In [6]:
seattle_listings.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,6606,Guesthouse in Seattle · ★4.60 · 1 bedroom · 1 ...,14942,Joyce,Other neighborhoods,Wallingford,47.65444,-122.33629,Entire home/apt,99,30,160,2023-08-05,0.93,2,177,1,str-opli-19-002622
1,9419,Rental unit in Seattle · ★4.72 · 1 bedroom · 1...,30559,Angielena,Other neighborhoods,Georgetown,47.55017,-122.31937,Private room,85,2,193,2023-09-04,1.21,9,365,23,STR-OPLI-19-003039
2,9531,Home in Seattle · ★4.96 · 2 bedrooms · 3 beds ...,31481,Cassie,West Seattle,Fairmount Park,47.55495,-122.38663,Entire home/apt,185,3,77,2023-09-09,0.54,2,318,12,STR-OPLI-19-002182
3,9534,Guest suite in Seattle · ★4.99 · 2 bedrooms · ...,31481,Cassie,West Seattle,Fairmount Park,47.55627,-122.38607,Entire home/apt,155,2,75,2023-05-28,0.53,2,230,7,STR-OPLI-19-002182
4,9596,Rental unit in Seattle · ★4.56 · 1 bedroom · 4...,14942,Joyce,Other neighborhoods,Wallingford,47.65608,-122.33602,Entire home/apt,130,30,97,2020-09-28,0.65,2,0,0,STR -OPLI-19-002622


In [10]:
from io import BytesIO

def read_csv_gz_from_url(url):
    """
    Fetches a gzipped CSV file from a URL, decompresses it, and reads it into a Pandas DataFrame.

    Args:
    - url: URL of the gzipped CSV file

    Returns:
    - Pandas DataFrame containing the data from the CSV file
    """
    response = requests.get(url)
    compressed_file = BytesIO(response.content)
    
    with gzip.open(compressed_file, 'rb') as gz_file:
        df = pd.read_csv(gz_file)
    
    return df

# Read data from each URL and assign to different DataFrame names
seattle_reviews = read_csv_gz_from_url('http://data.insideairbnb.com/united-states/wa/seattle/2023-09-18/data/reviews.csv.gz') 
la_reviews = read_csv_gz_from_url('http://data.insideairbnb.com/united-states/ca/los-angeles/2023-09-03/data/reviews.csv.gz') 

In [11]:
seattle_reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,6606,5664,2009-07-17,18085,Vivian,"The Urban Cottage is comfortable, beautiful, f..."
1,353110,1080517,2012-04-03,2019171,Allison & Jerry,"Simply a lovely place to stay. Quite, private..."
2,353110,1604096,2012-07-01,2086430,Mark,My girlfriend and I recently moved to Seattle ...
3,353110,1904384,2012-08-06,2886945,Davida,This is a great place in a great location. The...
4,353110,2563507,2012-10-09,3619132,Trish,My daughter and I stayed at Eric's place last ...


In [8]:
from textblob import TextBlob

#this can take a bit since TextBlob processing each text entry individually 
def add_sentiment_column(df):
    """
    Apply TextBlob's sentiment analysis to a specified text column in a DataFrame
    and create a new column for sentiment scores.

    Returns:
    - Updated DataFrame with the new column for sentiment scores
    """
    df['sentiment_score'] = df['comments'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)
    return df

seattle_reviews = add_sentiment_column(seattle_reviews)

# Display the updated DataFrame with sentiment scores
seattle_reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments,sentiment_score
0,6606,5664,2009-07-17,18085,Vivian,"The Urban Cottage is comfortable, beautiful, f...",0.330833
1,353110,1080517,2012-04-03,2019171,Allison & Jerry,"Simply a lovely place to stay. Quite, private...",0.253125
2,353110,1604096,2012-07-01,2086430,Mark,My girlfriend and I recently moved to Seattle ...,0.299667
3,353110,1904384,2012-08-06,2886945,Davida,This is a great place in a great location. The...,0.383300
4,353110,2563507,2012-10-09,3619132,Trish,My daughter and I stayed at Eric's place last ...,0.377778
